In [1]:
import math
from scipy.constants import Planck, alpha, e, epsilon_0, speed_of_light, m_p, m_n, c
from random import choices
import simpy as simp
import random as rm
import numpy as np

In [2]:
#separation energy function
def SEMF(Z, N):
    aV, aS, aC, aA, delta = 15.75, 17.8, 0.711, 23.7, 11.18
    Z, N = np.atleast_1d(Z), np.atleast_1d(N)
    A = Z + N
    sgn = np.zeros(Z.shape)
    if Z%2 ==0 and N%2 ==0:
        sgn=-1
    else:
        sgn = +1
    E = (aV - aS / A**(1/3) - aC * Z**2 / A**(4/3) -
         aA * (A-2*Z)**2/A**2 + sgn * delta/A**(3/2))
    if Z.shape[0] == 1:
        return float(E)
    return E

In [3]:
# neutron induced reactions:

# X(n,p)Y and X(n,n)Y reaction cross section function:
def sig_np(R,wav,e,V0):
    h = Planck/(2*math.pi)
    M = m_n
    k = ((2*M*e)**0.5)/h
    K = ((2*M*(e+V0))**0.5)/h
    sig2 = math.pi*((R+wav)**2)*((4*k*K)/((k+K)**2))
    C = 0.02
    a = 12
    eby = 300
    Q = eby - e
    w = C*math.exp(2*math.sqrt(a*(eby-e)))
    Fp = ((2*1.2)/h**2)*(((e+Q)*sig2*w*eby)-((e)*sig2*w*0))
    Fn = ((2*1)/h**2)*(((e)*sig2*w*eby)-((e+Q)*sig2*w*0))
    Fa = ((2*4)/h**2)*(((e+Q)*sig2*w*eby)-((e+Q)*sig2*w*0))
    sig = sig2*(Fp/(Fa+Fn+Fp))
    return sig,sig2

# X(n,cap)Y reaction cross section function
def sig_cap_av(R,l,e,V0):
    h = Planck/(2*math.pi)
    M = m_n
    k = ((2*M*e)**0.5)/h
    K = ((2*M*(e+V0))**0.5)/h
    x = k*R
    X = K*R
    vl = (x**(2*l))/(math.factorial(math.factorial(2*l - 1))**2)
    Di = 70
    rad = 0.17
    e1 = 50*((rad/Di)**2)
    sig = ((4*math.pi*(2*l + 1))/k*K)*(vl/(1+ ((e/e1)**0.5)*vl))
    return sig

# X(n,2n)Y reaction cross section function:
def sig_n2n(R,e,V0):
    Sn = SEMF(A/2, A/2)
    ec = e - Sn
    a = 12
    temp = (e/a)**0.5
    n2n = math.pi*(R**2)*(1 - (1+(ec/temp))*math.exp(-(ec/temp)))
    return n2n


# X+n => Y+b => Z+(p or n or 2n or al) reaction cross sections:
def sec_re(crosss,N,Z,e,R,react):
    h = Planck/(2*math.pi)
    eby = 300
    Sep = SEMF(Z,N)
    ec = eby-Sep
    a = 12
    temp = (eby/a)**0.5
    C = 0.02
    w = C*math.exp(2*math.sqrt(a*(eby-e)))
    Q = eby - e
    r = 1*(10**30)
    V = 0
    sig = math.pi*R*(1 - (V/e))
    Fp = ((2*1.2)/h**2)*(((e+Q)*sig*w*eby)-((e)*sig*w*0))
    Fn = ((2*1)/h**2)*(((e)*sig*w*eby)-((e+Q)*sig*w*0))
    Fnn = ((2*2)/h**2)*(((e+Q)*sig*w*eby)-((e+Q)*sig*w*0))
    Fa = ((2*4)/h**2)*(((e+Q)*sig*w*eby)-((e+Q)*sig*w*0))
    bindp = Fp/(Fp+Fn+Fa+Fnn)
    binda = Fa/(Fp+Fn+Fa+Fnn)
    bindnn = Fnn/(Fp+Fn+Fa+Fnn)
    
    n = crosss*(1-(1+(ec/temp))*math.exp(-(ec/temp)))
    nn = crosss*(((e*math.exp(-(e/temp))*bindnn*ec)-(e*math.exp(-(e/temp))*bindnn*0))/((e*math.exp(-(e/temp))*eby)-(e*math.exp(-(e/temp))*0)))
    al = crosss*(((e*math.exp(-(e/temp))*binda*ec)-(e*math.exp(-(e/temp))*binda*0))/((e*math.exp(-(e/temp))*eby)-(e*math.exp(-(e/temp))*0)))
    p = crosss*(((e*math.exp(-(e/temp))*bindp*ec)-(e*math.exp(-(e/temp))*bindp*0))/((e*math.exp(-(e/temp))*eby)-(e*math.exp(-(e/temp))*0)))
    #here the program takes a random weighted choice between the probability of the X(a,b)Y reaction
    #and the X+n => Y+b => Z+c reaction to see which reaction occurs
    sigg = ['(2)n','(2)nn','(2)al','(2)p',react]
    weights = [n/((R**2)*math.pi),nn/((R**2)*math.pi),al/((R**2)*math.pi),p/((R**2)*math.pi),crosss/((R**2)*math.pi)]
    win=choices(sigg, weights)
    return win, weights

In [4]:
#for protons

def sig_pn(R,e,A):
    h = Planck/(2*math.pi)
    Za = 1
    Zx = -90
    r = 1*(10**30)
    V = (Za*Zx*(2.7**2))/r
    sig = math.pi*R*(1 - (V/e))
    C = 0.02
    a = 12
    eby = 300
    w = C*math.exp(2*math.sqrt(a*(eby-e)))
    Q = eby - e
    Fp = ((2*1.2)/h**2)*(((e)*sig*w*eby)-((e)*sig*w*0))
    Fn = ((2*1)/h**2)*(((e+Q)*sig*w*eby)-((e+Q)*sig*w*0))
    Fnn = ((2*2)/h**2)*(((e+Q)*sig*w*eby)-((e+Q)*sig*w*0))
    Fa = ((2*4)/h**2)*(((e+Q)*sig*w*eby)-((e+Q)*sig*w*0))
    Fpn = ((2*2.2)/h**2)*(((e+Q)*sig*w*eby)-((e+Q)*sig*w*0))
    sigpn = sig*(Fn/(Fp+Fn+Fa+Fnn+Fpn))
    sigpa = sig*(Fa/(Fp+Fn+Fa+Fnn+Fpn))
    sigp2n = sig*(Fnn/(Fp+Fn+Fa+Fnn+Fpn))
    sigppn = sig*(Fpn/(Fp+Fn+Fa+Fnn+Fpn))
    return sigpn, sig, sigpa, sigp2n,sigppn

def sig_cap_av_p(R,e,V0):
    h = Planck/(2*math.pi)
    M = m_p
    l = 1
    k = ((2*M*e)**0.5)/h
    K = ((2*M*(e+V0))**0.5)/h
    x = k*R
    X = K*R
    vl = (x**(2*l))/(math.factorial(math.factorial(2*l - 1))**2)
    Di = 70
    rad = 0.17
    e1 = 50*((rad/Di)**2)
    sig = ((4*math.pi*(2*l + 1))/k*K)*(vl/(1+ ((e/e1)**0.5)*vl))
    return sig

In [5]:
Type = 'n'
A = 231    #amount of protons and neutron in the target nucleus (X)
counts = 1000    #amount of times the program will run
def collisions(env,Type,A,counts):
    h = Planck/(2*math.pi)
    testNum=1
    r_0 = 1.33
    R = r_0*(A**(1/3))    #radius
    V0 = 20    #well depth
    #for neutron induced reactions
    if Type == 'n':
        M = m_n    #mass of the incident particle
        react = ['np','cap','n2n','scat','a','nn']
        #for keeping score of reactions that have occured
        n2n = 0
        a = 0
        scat = 0
        nn = 0
        cap = 0
        np = 0
        #for keeping score of secondary reactions that have occured
        sec_reactions = 0
        for i in range(counts+1):
            print('Test %d' % testNum)
            e=rm.randint(1000,2000)/100    #entrance channel energy
            l = 1    #linear momentum
            nl = 0
            f = 0.003    #time in seconds
            k = ((2*M*e)**0.5)/h    #channel wave number
            wav = 1/k    #channel wavelength
            yield env.timeout(1)
            #cross sections:
            sig__np, sig__nn = sig_np(R,wav,e,V0)
            sig_cap__av = sig_cap_av(R,l,e,V0)
            sig__n2n = sig_n2n(R,e,V0)
            sig__sc = math.pi*(wav**2)*(2*l +1)*(abs(1-nl)**2)
            sig__a = (2*math.pi)*(2*l +1)*(1-abs(nl))
            #here the program divides the cross section by the target area ((R^2)*pi)
            cross = [sig__np/(math.pi*(R**2)),sig_cap__av/(math.pi*(R**2)),sig__n2n/(math.pi*(R**2)),sig__sc/(math.pi*(R**2)),sig__a/(math.pi*(R**2)),sig__nn/(math.pi*(R**2))]
            win=choices(react, cross)
            if win==['n2n']:
                n2n+=1
                wiin = True
                winner,croos = sec_re(sig__n2n,A/2,A/2,e,R,'n2n')
            elif win==['a']:
                a+=1
                wiin = False
            elif win==['scat']:
                scat+=1
                wiin = True
                winner,croos = sec_re(sig__sc,A/2,A/2,e,R,'scat')
            elif win==['nn']:
                nn+=1
                wiin = True
                winner,croos = sec_re(sig__nn,A/2,A/2,e,R,'nn')
            elif win==['cap']:
                cap+=1
                wiin = False
            elif win==['np']:
                np+=1
                wiin = True
                winner,croos = sec_re(sig__sc,A/2,A/2,e,R,'np')
            print(e)
            print(cross)
            testNum+=1
            print('n2n',n2n,'scat',scat,'a',a,'nn',nn,'cap',cap,'np',np)
            if wiin==True and win!=winner:
                print(croos)
                print('The reaction is {},{}'.format(win,winner))
                sec_reactions+=1
            else:
                print('The reaction is {}'.format(win))
            print('there have been',sec_reactions,'secondary reactions\n')
    #for proton induced reactions
    elif Type=='p':
        react = ['pn','pp','pa','p2n','ppn','cap']
        pn = 0
        pp = 0
        pa = 0
        p2n = 0
        ppn = 0
        cap = 0
        for i in range(counts):
            print('Test %d' % testNum)
            e=rm.randint(1000,1500)/10
            yield env.timeout(1)
            sig__pn,sig__pp,sig__pa,sig__p2n,sig__ppn = sig_pn(R,e,A)
            sig__cap = sig_cap_av_p(R,e,V0)
            cross = [sig__pn/(math.pi*(R**2)),sig__pp/(math.pi*(R**2)),sig__pa/(math.pi*(R**2)),sig__p2n/(math.pi*(R**2)),sig__ppn/(math.pi*(R**2)),sig__cap/(math.pi*(R**2))]
            win=choices(react, cross)
            if win == ['pn']:
                pn+=1
            elif win == ['pp']:
                pp+=1
            elif win == ['pa']:
                pa+=1
            elif win == ['ppn']:
                ppn+=1
            elif win == ['cap']:
                cap+=1
            else:
                p2n+=1
            testNum+=1
            print(win)
            print(e)
            print(pn,pp,pa,p2n,ppn,cap)
            print(cross,'\n')
env = simp.Environment()
env.process(collisions(env,Type,A,counts))
env.run(until=counts+1)

Test 1
16.24
[0.21943588645432952, 0.0011470453728464603, 0.9983319653992972, 9.209094759827688e-45, 0.09009623619142512, 0.960787837957701]
n2n 1 scat 0 a 0 nn 0 cap 0 np 0
[0.9983319653992972, 0.2695930728663141, 0.5391861457326282, 0.16175584371978846, 0.9983319653992972]
The reaction is ['n2n'],['(2)al']
there have been 1 secondary reactions

Test 2
16.72
[0.21971286060946235, 0.0011214698203878977, 0.9987043952188522, 8.944718833708233e-45, 0.09009623619142512, 0.9622935043893097]
n2n 1 scat 0 a 0 nn 1 cap 0 np 0
[0.9622935043893097, 0.25980381684220005, 0.5196076336844001, 0.15588229010531995, 0.9622935043893097]
The reaction is ['nn'],['(2)n']
there have been 2 secondary reactions

Test 3
14.51
[0.2182422093298489, 0.0012527880292511587, 0.9956705467075312, 1.030707780148874e-44, 0.09009623619142512, 0.9545126139220568]
n2n 1 scat 0 a 0 nn 1 cap 0 np 1
The reaction is ['np']
there have been 2 secondary reactions

Test 4
16.62
[0.21965686737250037, 0.0011266802416436108, 0.998634

n2n 60 scat 0 a 5 nn 48 cap 0 np 10
[0.9996951804084078, 0.2695388536726859, 0.5390777073453717, 0.16172331220361144, 0.9996951804084078]
The reaction is ['n2n'],['(2)p']
there have been 82 secondary reactions

Test 124
12.29
[0.21611490488563456, 0.0014307200630920714, 0.9834766018786527, 1.2168893319739757e-44, 0.09009623619142512, 0.9438758438962065]
n2n 61 scat 0 a 5 nn 48 cap 0 np 10
[0.9834766018786527, 0.2660644003833615, 0.532128800766723, 0.15963864023001692, 0.9834766018786527]
The reaction is ['n2n'],['(2)n']
there have been 83 secondary reactions

Test 125
15.58
[0.2190190656213937, 0.0011846990665185166, 0.9976196589345034, 9.599210455686884e-45, 0.09009623619142512, 0.9585612761437654]
n2n 61 scat 0 a 5 nn 49 cap 0 np 10
The reaction is ['nn']
there have been 83 secondary reactions

Test 126
12.46
[0.21630795901204802, 0.0014149097625870191, 0.9851646002609465, 1.200286508022485e-44, 0.09009623619142512, 0.9448211478557917]
n2n 62 scat 0 a 5 nn 49 cap 0 np 10
[0.985164600

there have been 127 secondary reactions

Test 189
11.93
[0.21568639772080875, 0.0014656542892773911, 0.9791769354259265, 1.2536102170964097e-44, 0.09009623619142512, 0.9417886643590858]
n2n 93 scat 0 a 6 nn 75 cap 0 np 15
[0.9791769354259265, 0.2649450997424169, 0.5298901994848338, 0.15896705984545015, 0.9791769354259265]
The reaction is ['n2n'],['(2)p']
there have been 128 secondary reactions

Test 190
17.0
[0.21986505777371915, 0.0011071958986499095, 0.9988794794395831, 8.797394052917742e-45, 0.09009623619142512, 0.9631311003032086]
n2n 93 scat 0 a 6 nn 75 cap 0 np 16
[8.797394052917742e-45, 2.3748498513048322e-45, 4.7496997026096645e-45, 1.424909910782899e-45, 8.797394052917742e-45]
The reaction is ['np'],['(2)n']
there have been 129 secondary reactions

Test 191
17.83
[0.2202790961216012, 0.0010674298789553434, 0.9992647232711955, 8.38786869879987e-45, 0.09009623619142512, 0.9654526839820723]
n2n 93 scat 0 a 6 nn 76 cap 0 np 16
[0.9654526839820723, 0.2605238927170247, 0.52104778543


Test 287
18.59
[0.22061423919282916, 0.0010340237815596356, 0.9994944343109047, 8.04495421729971e-45, 0.09009623619142512, 0.967387310687245]
n2n 137 scat 0 a 11 nn 114 cap 0 np 25
[0.9994944343109047, 0.269615846966036, 0.539231693932072, 0.16176950817962166, 0.9994944343109047]
The reaction is ['n2n'],['(2)n']
there have been 194 secondary reactions

Test 288
12.54
[0.21639683392052572, 0.0014076146089647842, 0.9858936856473254, 1.192629177827764e-44, 0.09009623619142512, 0.9452574367038432]
n2n 137 scat 0 a 11 nn 115 cap 0 np 25
[0.9452574367038432, 0.25569536877603655, 0.5113907375520731, 0.1534172212656219, 0.9452574367038432]
The reaction is ['nn'],['(2)p']
there have been 195 secondary reactions

Test 289
13.9
[0.21773465731203906, 0.001296156901288208, 0.9938312977242392, 1.0759402798532491e-44, 0.09009623619142512, 0.9519238253983843]
n2n 137 scat 0 a 11 nn 116 cap 0 np 25
[0.9519238253983843, 0.2573375575906836, 0.5146751151813672, 0.15440253455441016, 0.9519238253983843]
Th


Test 403
19.44
[0.22094532251405388, 0.0009996425191009085, 0.9996635307715278, 7.693194387839591e-45, 0.09009623619142512, 0.9693607783099921]
n2n 189 scat 0 a 18 nn 160 cap 0 np 36
The reaction is ['n2n']
there have been 266 secondary reactions

Test 404
17.72
[0.22022724195007606, 0.0010724934590999682, 0.9992230875847934, 8.439937861151336e-45, 0.09009623619142512, 0.9651581226929835]
n2n 190 scat 0 a 18 nn 160 cap 0 np 36
[0.9992230875847934, 0.2696503329784269, 0.5393006659568538, 0.16179019978705614, 0.9992230875847934]
The reaction is ['n2n'],['(2)n']
there have been 267 secondary reactions

Test 405
16.14
[0.21937550057443153, 0.001152558748117224, 0.9982407213043093, 9.266152348178537e-45, 0.09009623619142512, 0.9604625040982901]
n2n 190 scat 0 a 18 nn 161 cap 0 np 36
[0.9604625040982901, 0.25937858909065187, 0.5187571781813037, 0.15562715345439115, 0.9604625040982901]
The reaction is ['nn'],['(2)n']
there have been 268 secondary reactions

Test 406
16.0
[0.21928934010152287

...

15.16
[0.2187300319315744, 0.0012103131715079268, 0.9969996438360635, 9.865151642453936e-45, 0.09009623619142512, 0.957041103048163]
n2n 265 scat 0 a 26 nn 227 cap 0 np 53
[0.9969996438360635, 0.26936696648811537, 0.5387339329762307, 0.16162017989286917, 0.9969996438360635]
The reaction is ['n2n'],['(2)n']
there have been 362 secondary reactions

Test 572
17.37
[0.22005624081465958, 0.0010890159941508058, 0.9990728644448172, 8.609999936649488e-45, 0.09009623619142512, 0.9641947571494989]
n2n 266 scat 0 a 26 nn 227 cap 0 np 53
[0.9990728644448172, 0.26965313202358615, 0.5393062640471723, 0.16179187921415164, 0.9990728644448172]
The reaction is ['n2n'],['(2)nn']
there have been 363 secondary reactions

Test 573
15.4
[0.21889757473537869, 0.0011955106185839524, 0.9973727352443421, 9.71140901945465e-45, 0.09009623619142512, 0.9579201089947655]
n2n 266 scat 0 a 26 nn 228 cap 0 np 53
[0.9579201089947655, 0.25877999879744706, 0.5175599975948941, 0.15526799927846824, 0.9579201089947655]
The re

there have been 419 secondary reactions

Test 663
13.51
[0.21738209475935086, 0.0013258805281325965, 0.9922230416468174, 1.1069999918549342e-44, 0.09009623619142512, 0.9501469442355172]
n2n 302 scat 0 a 30 nn 267 cap 0 np 64
The reaction is ['n2n']
there have been 419 secondary reactions

Test 664
13.82
[0.21766420260795583, 0.0013021208298571864, 0.9935333016267622, 1.0821685882749754e-44, 0.09009623619142512, 0.9515674315234807]
n2n 303 scat 0 a 30 nn 267 cap 0 np 64
The reaction is ['n2n']
there have been 419 secondary reactions

Test 665
16.47
[0.2195712126185225, 0.001134610531025836, 0.9985230646443803, 9.080491736466398e-45, 0.09009623619142512, 0.9615206376575616]
n2n 304 scat 0 a 30 nn 267 cap 0 np 64
[0.9985230646443803, 0.2696161830868915, 0.539232366173783, 0.16176970985213487, 0.9985230646443803]
The reaction is ['n2n'],['(2)al']
there have been 420 secondary reactions

Test 666
15.11
[0.2186943049510438, 0.001213454448462182, 0.9969149470129002, 9.897796088656626e-45, 0.0

The reaction is ['nn'],['(2)n']
there have been 473 secondary reactions

Test 758
14.94
[0.21857065174358456, 0.0012242874019015183, 0.9966071418332191, 1.0010421613092478e-44, 0.09009623619142512, 0.9562101729362251]
n2n 345 scat 0 a 33 nn 310 cap 0 np 70
[0.9562101729362251, 0.2583726976460913, 0.5167453952921826, 0.1550236185876548, 0.9562101729362251]
The reaction is ['nn'],['(2)n']
there have been 474 secondary reactions

Test 759
10.95
[0.21436807208569733, 0.0015721310138173492, 0.9600459643784164, 1.3658054694027552e-44, 0.09009623619142512, 0.9354486745639619]
n2n 345 scat 0 a 33 nn 311 cap 0 np 70
The reaction is ['nn']
there have been 474 secondary reactions

Test 760
18.08
[0.22039371353480694, 0.0010561426267266857, 0.9993507020299622, 8.27188600108416e-45, 0.09009623619142512, 0.9661080873750224]
n2n 345 scat 0 a 33 nn 312 cap 0 np 70
The reaction is ['nn']
there have been 474 secondary reactions

Test 761
15.57
[0.21901240750729462, 0.0011852933497571861, 0.9976066182449

n2n 384 scat 0 a 36 nn 351 cap 0 np 80
[7.817861939341431e-45, 2.108363055388952e-45, 4.216726110777904e-45, 1.2650178332333715e-45, 7.817861939341431e-45]
The reaction is ['np'],['(2)al']
there have been 533 secondary reactions

Test 852
12.76
[0.21663494013280513, 0.001388013850706846, 0.9877064743156533, 1.1720666057962512e-44, 0.09009623619142512, 0.9464299123424182]
n2n 384 scat 0 a 36 nn 352 cap 0 np 80
The reaction is ['nn']
there have been 533 secondary reactions

Test 853
12.52
[0.21637473169677654, 0.0014094298572030284, 0.9857151144400692, 1.1945343362587986e-44, 0.09009623619142512, 0.9451488696883752]
n2n 385 scat 0 a 36 nn 352 cap 0 np 80
The reaction is ['n2n']
there have been 533 secondary reactions

Test 854
10.48
[0.21364523828780038, 0.0016301168117003164, 0.9447369157058463, 1.4270581956068854e-44, 0.09009623619142512, 0.9320154828506243]
n2n 385 scat 0 a 36 nn 352 cap 0 np 81
[1.4270581956068854e-44, 3.8639049653235885e-45, 7.727809930647177e-45, 2.3183429791941532

n2n 415 scat 0 a 41 nn 388 cap 0 np 92
[0.9484558979459656, 0.25679223367941106, 0.5135844673588221, 0.15407534020764665, 0.9484558979459656]
The reaction is ['n2n'],['(2)p']
there have been 591 secondary reactions

Test 937
14.65
[0.218351681998536, 0.0012433303141776723, 0.9960029313186605, 1.0208580129665642e-44, 0.09009623619142512, 0.9550763223860964]
n2n 415 scat 0 a 41 nn 389 cap 0 np 92
[0.9550763223860964, 0.2581007400250919, 0.5162014800501838, 0.1548604440150551, 0.9550763223860964]
The reaction is ['nn'],['(2)al']
there have been 592 secondary reactions

Test 938
19.18
[0.22084861787980556, 0.0010098483046501968, 0.9996193846681148, 7.79748169445264e-45, 0.09009623619142512, 0.9687770010650871]
n2n 415 scat 0 a 41 nn 390 cap 0 np 92
[0.9687770010650871, 0.26125899754612253, 0.5225179950922451, 0.1567553985276735, 0.9687770010650871]
The reaction is ['nn'],['(2)nn']
there have been 593 secondary reactions

Test 939
17.09
[0.2199125812307698, 0.0011027038265908975, 0.99893020